In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta  # Import necessary modules


In [2]:
df = yf.download(tickers = 'BTC-USD', start = '2019-04-15',end = str(pd.Timestamp.today()).split(' ')[0])
df = df.drop(columns=['Open','High','Low','Close','Volume'])
df = df.dropna()

[*********************100%%**********************]  1 of 1 completed


In [3]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

In [4]:
sequence_length = 10
X, y = [], []

In [5]:
for i in range(len(df_scaled) - sequence_length):
    X.append(df_scaled[i:i+sequence_length, 0])
    y.append(df_scaled[i+sequence_length, 0])

In [6]:
X, y = np.array(X), np.array(y)

In [7]:
X = np.reshape(X, (X.shape[0], X.shape[1], 1))


In [8]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [9]:
model.fit(X, y, epochs=20, batch_size=32)

Epoch 1/20

56/56 [==============================] - 3s 4ms/step - loss: 0.0191
Epoch 2/20
56/56 [==============================] - 0s 4ms/step - loss: 0.0013
Epoch 3/20
56/56 [==============================] - 0s 4ms/step - loss: 0.0013
Epoch 4/20
56/56 [==============================] - 0s 5ms/step - loss: 0.0013
Epoch 5/20
56/56 [==============================] - 0s 4ms/step - loss: 0.0013
Epoch 6/20
56/56 [==============================] - 0s 5ms/step - loss: 0.0013
Epoch 7/20
56/56 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 8/20
56/56 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 9/20
56/56 [==============================] - 0s 4ms/step - loss: 0.0011
Epoch 10/20
56/56 [==============================] - 0s 4ms/step - loss: 0.0010
Epoch 11/20
56/56 [==============================] - 0s 5ms/step - loss: 0.0010
Epoch 12/20
56/56 [==============================] - 0s 5ms/step - loss: 0.0010
Epoch 13/20
56/56 [=============================

In [10]:
last_sequence = df_scaled[-sequence_length:].reshape((1, sequence_length, 1))
future_predictions = []

In [11]:
for i in range(7):
    next_day_prediction = model.predict(last_sequence)
    future_predictions.append(next_day_prediction[0, 0])
    last_sequence = np.roll(last_sequence, -1, axis=1)
    last_sequence[0, -1, 0] = next_day_prediction[0, 0]

1/1 [==============================] - 0s 15ms/step


In [12]:
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))


In [13]:
tomorrow_date = datetime.now() + timedelta(days=1)

In [14]:
print("Predicted Prices for the Next 7 Days:")
for i in range(7):
    print(f"{tomorrow_date.strftime('%Y-%m-%d')}: {future_predictions[i][0]}")
    tomorrow_date += timedelta(days=1)

Predicted Prices for the Next 7 Days:
2024-03-13: 68279.78125
2024-03-14: 68332.6328125
2024-03-15: 67893.125
2024-03-16: 67391.0078125
2024-03-17: 66316.15625
2024-03-18: 65221.23046875
2024-03-19: 64052.51171875
